# Segmenting and Clustering Neighborhoods in Toronto (1st of 3 parts)

A peer-graded assignment on Coursera made by M. O.

## Assignment

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install folium

In [3]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import folium
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize 

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

## Scrap content from Wikipedia article

In [370]:
source = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1012118802").text
soup = BeautifulSoup(source,'html.parser')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["PostalCode"] = df["PostalCode"].str.replace("\n","")

In [248]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source,'html.parser')

table_contents=[]
table = soup.find("table")
table_rows = table.tbody.find_all("tr")

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [252]:
print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'PostalCode': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'PostalCode': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'PostalCode': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'PostalCode': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'PostalCode': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'PostalCode': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'PostalCode': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'PostalCode': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'PostalCode': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'PostalCode': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'PostalCode': 'M9

In [369]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [254]:
print("Shape: ", df.shape)

Shape:  (103, 3)


# Latitude and Longitude coordinates for neighborhoods (Part 2 of 3)

In [255]:
import geocoder # import geocoder

In [256]:
df_geo_coor = pd.read_csv("C:/Users/Michael/Desktop/Data Science/Foursquare - Capstone/Geospatial_Coordinates.csv")
df_geo_coor.shape

(103, 3)

In [257]:
df_toronto = pd.merge(df, df_geo_coor, how='right', left_on = df['PostalCode'], right_on = df_geo_coor['Postal Code'])
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,key_0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Explore and Cluster (Part 3 of 3)

Convert Toronto to its latitude and longitude coordinates

In [298]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [299]:
#Map the boroughs with "Toronto" onto map
df_toronto_centr = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_centr.head()

,key_0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [300]:
map_toronto_centr = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(df_toronto_centr['Latitude'], df_toronto_centr['Longitude'], df_toronto_centr['Borough'], df_toronto_centr['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_toronto_centr)
    
map_toronto_centr

#### Foursquare credentials

In [301]:
CLIENT_ID = 'KZMA35V044ZK5MXXV4HP54C3540NFU2XIQGRY0NQM3XFO40S' # your Foursquare ID
CLIENT_SECRET = 'QOKBVTPE55003ESJLC4FO0T5CUPS3MOXGYFPVHXDY02UQHTS' # your Foursquare Secret
ACCESS_TOKEN = '1W1CSZZCVKW3AXXOFZHGY2JAB34YCIDRJDJYJH5RSGUE5TB1' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KZMA35V044ZK5MXXV4HP54C3540NFU2XIQGRY0NQM3XFO40S
CLIENT_SECRET:QOKBVTPE55003ESJLC4FO0T5CUPS3MOXGYFPVHXDY02UQHTS


Location wise, I like St. James Town. It's a neighborhood in the city, but close to a river and lots of green spaces --> let's explore it.

### Exploring St. James Town

In [302]:
neighborhood_latitude = df_toronto_centr.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto_centr.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto_centr.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of St. James Town are 43.6514939, -79.3754179.


#### Let's get the top 100 venues that are in St. James Town within a radius of 500 meters.


In [303]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)

results = requests.get(url).json()



In [304]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [305]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-305-4e8e655a677d>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Versus Coffee,Coffee Shop,43.651213,-79.375236
1,60 Adelaide St. East,Office,43.651547,-79.375713
2,Courthouse Square,Park,43.651611,-79.375283
3,Fresh and Prestige,Laundry Service,43.651255,-79.375485
4,The Spire,Residential Building (Apartment / Condo),43.651622,-79.375289


In [306]:
print('{} venues were returned from Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned from Foursquare.


### Let's get the same information for all of the neighborhoods that have "Toronto" in the borough name

In [351]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [352]:
# type your answer here
df_toronto_centr_venues = getNearbyVenues(names=df_toronto_centr['Neighborhood'],
                                   latitudes=df_toronto_centr['Latitude'],
                                   longitudes=df_toronto_centr['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [353]:
df_toronto_centr_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,47,47,47,47,47,47
"Brockton, Parkdale Village, Exhibition Place",21,21,21,21,21,21
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,62,62,62,62,62,62
Christie,14,14,14,14,14,14
Church and Wellesley,66,66,66,66,66,66
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,27,27,27,27,27,27
Davisville North,10,10,10,10,10,10


In [354]:
# one hot encoding
df_toronto_centr_onehot = pd.get_dummies(df_toronto_centr_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_toronto_centr_onehot['Neighborhood'] = df_toronto_centr_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_toronto_centr_onehot.columns[-1]] + list(df_toronto_centr_onehot.columns[:-1])
df_toronto_centr_onehot = df_toronto_centr_onehot[fixed_columns]

df_toronto_centr_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [355]:
df_toronto_centr_grouped = df_toronto_centr_onehot.groupby('Neighborhood').mean().reset_index()
df_toronto_centr_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.042553,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.066667,0.066667,0.133333,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.016129,0.000000,0.000000,0.016129,0.000000
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.015152,0.015152,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.015152,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.015152
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.037037,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


#### Let's print each neighborhood along with the top 5 most common venues


In [356]:
num_top_venues = 5

for hood in df_toronto_centr_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = df_toronto_centr_grouped[df_toronto_centr_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.09
2  Sandwich Place  0.06
3          Bakery  0.06
4  Farmers Market  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0  Breakfast Spot  0.10
1            Café  0.10
2     Coffee Shop  0.10
3  Sandwich Place  0.10
4   Grocery Store  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0     Airport Terminal  0.13
1          Coffee Shop  0.07
2                Plane  0.07
3  Rental Car Location  0.07
4     Sculpture Garden  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1      Sandwich Place  0.08
2    Sushi Restaurant  0.06
3                Café  0.05
4  Italian Restaurant  0.05


----Christie----
           venue  freq
0  Grocery Store  0.29
1           Café  0.21
2           Park  0.14
3      Nightclub  0.07
4     Baby Stor

#### Let's put that into a _pandas_ dataframe

In [357]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [358]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_toronto_centr_grouped['Neighborhood']

for ind in np.arange(df_toronto_centr_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_toronto_centr_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Sandwich Place,Bakery,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,French Restaurant,Diner
1,"Brockton, Parkdale Village, Exhibition Place",Sandwich Place,Coffee Shop,Breakfast Spot,Café,Convenience Store,Restaurant,Japanese Restaurant,Italian Restaurant,Intersection,Bar
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Coffee Shop,Sculpture Garden,Boutique,Rental Car Location,Bar,Plane,Harbor / Marina,Boat or Ferry,Airport Service
3,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Japanese Restaurant,Café,Restaurant,Salad Place,Bank,Burger Joint
4,Christie,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Restaurant,Coffee Shop,Nightclub,Discount Store,Diner


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [359]:
# set number of clusters
kclusters = 5

df_toronto_centr_grouped_clustering = df_toronto_centr_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_toronto_centr_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [360]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_toronto_centr_merged = df_toronto_centr

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
df_toronto_centr_merged = df_toronto_centr_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_toronto_centr_merged.head() # check the last columns!

,key_0,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Café,Greek Restaurant,Event Space,Bank,Farmers Market,Spa
1,M5B,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Hotel,Middle Eastern Restaurant,Pizza Place,Cosmetics Shop,Japanese Restaurant,Sandwich Place,Falafel Restaurant
2,M5C,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Cocktail Bar,Café,Italian Restaurant,Restaurant,Clothing Store,Cosmetics Shop,Gym,Gastropub,Theater
3,M4E,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Playground,Pub,Wine Shop,Cosmetics Shop,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
4,M5E,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Sandwich Place,Bakery,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,French Restaurant,Diner


In [361]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto_centr_merged['Latitude'], df_toronto_centr_merged['Longitude'], df_toronto_centr_merged['Neighborhood'], df_toronto_centr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5 different clusters? 

### Cluster 1

In [363]:
df_toronto_centr_merged.loc[df_toronto_centr_merged['Cluster Labels'] == 0, df_toronto_centr_merged.columns[[1] + list(range(5, df_toronto_centr_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,M4J,-79.338106,0,Park,Intersection,Convenience Store,Cuban Restaurant,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop
33,M4W,-79.377529,0,Park,Trail,Playground,Convenience Store,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


### Cluster 2

In [364]:
df_toronto_centr_merged.loc[df_toronto_centr_merged['Cluster Labels'] == 1, df_toronto_centr_merged.columns[[1] + list(range(5, df_toronto_centr_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Café,Greek Restaurant,Event Space,Bank,Farmers Market,Spa
1,M5B,-79.378937,1,Coffee Shop,Clothing Store,Café,Hotel,Middle Eastern Restaurant,Pizza Place,Cosmetics Shop,Japanese Restaurant,Sandwich Place,Falafel Restaurant
2,M5C,-79.375418,1,Coffee Shop,Cocktail Bar,Café,Italian Restaurant,Restaurant,Clothing Store,Cosmetics Shop,Gym,Gastropub,Theater
3,M4E,-79.293031,1,Health Food Store,Playground,Pub,Wine Shop,Cosmetics Shop,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
4,M5E,-79.373306,1,Coffee Shop,Cocktail Bar,Sandwich Place,Bakery,Farmers Market,Seafood Restaurant,Beer Bar,Vegetarian / Vegan Restaurant,French Restaurant,Diner
5,M5G,-79.387383,1,Coffee Shop,Sandwich Place,Sushi Restaurant,Italian Restaurant,Japanese Restaurant,Café,Restaurant,Salad Place,Bank,Burger Joint
6,M6G,-79.422564,1,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Restaurant,Coffee Shop,Nightclub,Discount Store,Diner
7,M5H,-79.384568,1,Coffee Shop,Café,Sandwich Place,Clothing Store,Gym,Restaurant,Sushi Restaurant,Deli / Bodega,Bank,Hotel
8,M6H,-79.442259,1,Supermarket,Bar,Brewery,Bakery,Bank,Café,Middle Eastern Restaurant,Pharmacy,Liquor Store,Grocery Store
10,M5J,-79.381752,1,Coffee Shop,Hotel,Café,Scenic Lookout,Pizza Place,Aquarium,Gym,Sandwich Place,Bank,Boat or Ferry


### Cluster 3

In [366]:
df_toronto_centr_merged.loc[df_toronto_centr_merged['Cluster Labels'] == 2, df_toronto_centr_merged.columns[[1] + list(range(5, df_toronto_centr_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,M4T,-79.38316,2,Restaurant,Wine Shop,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


### Cluster 4

In [367]:
df_toronto_centr_merged.loc[df_toronto_centr_merged['Cluster Labels'] == 3, df_toronto_centr_merged.columns[[1] + list(range(5, df_toronto_centr_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5N,-79.416936,3,Garden,Home Service,Wine Shop,Creperie,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop


### Cluster 5

In [368]:
df_toronto_centr_merged.loc[df_toronto_centr_merged['Cluster Labels'] == 4, df_toronto_centr_merged.columns[[1] + list(range(5, df_toronto_centr_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,M4N,-79.388790,4,Park,Business Service,Bus Line,Swim School,College Arts Building,College Gym,Dog Run,Distribution Center,Discount Store,Diner
21,M5P,-79.411307,4,Park,Trail,Jewelry Store,Bus Line,Sushi Restaurant,College Arts Building,Cuban Restaurant,Dog Run,Distribution Center,Clothing Store
